### Refine Chain

* Updating a rolling summary be interating over the documents in a sequence

In [1]:
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFLoader
import os
from dotenv import load_dotenv

load_dotenv()

groq_api_key = os.getenv('GROQ_API_KEY') # API Key

llm = ChatGroq(groq_api_key=groq_api_key, model_name='llama3-8b-8192')

load = PyPDFLoader('./data/apjspeech.pdf')

docs = load.load_and_split()

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

final_document = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100).split_documents(docs)
final_document

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': './data/apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the futur

In [12]:
from langchain import PromptTemplate

chunk_prompts = """
Please summarize the below speech:
Speech: {text}
"""

question_prompt = PromptTemplate(input_variables=['text'], template=chunk_prompts)

In [14]:
final_prompt = """
Provide the final summary of the entire speech with these important points.
Add a motivational title, start the precise summary with an introduction and provide the summary in number points for the speech.
Speech: {text}
Existing summary: {existing_answer}
"""
refine_prompt = PromptTemplate(input_variables=['text', 'existing_answer'], template=final_prompt)

In [15]:
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(llm=llm, chain_type="refine", question_prompt=question_prompt, refine_prompt=refine_prompt, verbose=True)

output = chain.run(final_document)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_17552\2306217387.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = chain.run(final_document)




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Please summarize the below speech:
Speech: A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have many